# **Diplomado IA: Inteligencia Artificial II - Parte 1**. <br> Laboratorio 1: Redes Relacionales y Transformers
---
---

**Profesor:**
- Felipe del Río

**Ayudante:**
- Bianca del Solar
---
---

# **Instrucciones Generales**

El siguiente práctico será **individual**. Solo uno debe realizar la entrega. El formato de entregar es el **archivo .ipynb con todas las celdas ejecutadas**. Todas las preguntas deben ser respondida en celdas de texto. No se aceptará el _output_ de una celda de código como respuesta.

**Nombre:** COMPLETAR

**Fecha de entrega: Viernes 30 de Junio.**

El siguiente práctico cuenta con 2 secciones donde cada una contendrá 1 o más actividades a realizar. Algunas actividades correspondrán a escribir código y otras a responder preguntas.

**Importante.** Para facilitar su ejecución, cada sección puede ser ejecutada independientemente.

Se recomienda **fuertemente** revisar las secciones donde se entrega código porque algunas actividades de código pueden reutilizar el mismo código pero con cambios en algunas líneas.



# **Agenda**

>[Diplomado IA: Inteligencia Artificial II - Parte 1.  Laboratorio 1: Redes Relacionales y Transformers](#scrollTo=tHopPtVaNF1K)

>[Instrucciones Generales](#scrollTo=uIdAKAdELPSl)

>[Agenda](#scrollTo=kEloa5uXLIPK)

>[Parte III: Inspeccionando a CLIP](#scrollTo=ZS3cYFT2TWB9)

>>[Preámbulo](#scrollTo=0dBeU4b818s4)

>>[Cargamos el Modelo](#scrollTo=1D5aBia_ucDJ)

>>[Classificación zero-shot utilizando CLIP](#scrollTo=IPrwC1iF8K0j)

>>>[Dataset Food101](#scrollTo=IPrwC1iF8K0j)

>>>[Actividad 3](#scrollTo=3tABpd-l6-xH)

>>>[Dataset Stanford Cars](#scrollTo=qwbkd4RaAqYt)

>>>[Actividad](#scrollTo=CO9kxJAf6SDO)



# Parte III: Inspeccionando a CLIP

Luego de haber visualizado el funcionamento interno de un transformer, utilizaremos CLIP para ver como se puede aprovechar al máximo esta arquitectura en problemas multi-modales, en este caso, que mezclan texto con imágenes.

Como vimos en clases CLIP es un modelo que nos permite codificar tanto imágenes como texto en vectores de representación comparables entre ellos. Este nos permite ver cual es el texto que más se relaciona con una imágen determinada, y podemos aprovechar este mecanismo para clasificar imágenes con una mayor flexibilidad que en un modelo de clasificación tradicional. En esta tercera parte del laboratorio exploraremos como utilizar CLIP con este fin.

<small>Este notebook fue basado en [el provisto por OpenAI](https://colab.research.google.com/github/openai/clip/blob/master/notebooks/Interacting_with_CLIP.ipynb#scrollTo=uLFS29hnhlY4) con el fin de interacturar con CLIP</small>


## Preámbulo

Primero debemos descargar, instalar e importar las distintas librerías que utilizaremos para este laboratorio.

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import os
import skimage
import random
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch
from pkg_resources import packaging

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
def bold(text):
    return '\033[1m' + text + '\033[0m'

## Cargamos el Modelo

Luego cargaremos el modelo a utilizar. Utilizando la función `clip.available_models()` podemos listar los diferentes modelos displonibles. Pueden ver más detalles de lo que significa cada modelo en el [paper](https://arxiv.org/pdf/2103.00020.pdf) de CLIP.

Pero, a grandes razgos, **RN** corresponde a un backbone de ResNet para el encoder visual, RN50 correspondría a una ResNet50, y cuando tiene el sufijo `xN`, `x4` por ejemplo, significa que el modelo está escalado para utilizar `N` veces más computo.

Mientras que **ViT** corresponde a un backbone de Vision Transformer. El símbolo `-B` o `-L` corresponde al tamaño del modelo, Base y Large respectivamente y el sufijo `/32` corresponde a que los patches son de `32x32`.

Para este laboratorio usaremos la versión `ViT-B/32`, que está basado en su totalidad en Transformers.



In [ ]:
import clip

clip.available_models()

Primero carguemos un modelo, no tan grande, para utilizar en esta actividad.

El código a continuación descargará los pesos del modelo preentrenado de forma automática. Una vez cargados, podemos veamos algunas características relevantes del modelo.

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model, preprocess = clip.load("ViT-B/32")
model.to(device).eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Número de parámetros:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Resolución de Entrada:", input_resolution)
print("Tamaño del contexto:", context_length)
print(f"Tamaño del vocabulario: {vocab_size:,}")

Podemos acceder al preprocesamiento que CLIP realiza sobre las imágenes, en la variable `preprocess`. Este se basa en las transformaciones de PyTorch para transformar una imágen a un tensor normalizado que el modelo puede recibir (recuerden la clase anterior de data augmentation).

In [ ]:
preprocess

# Classificación zero-shot utilizando CLIP

Ya teniendo el modelo cargado, veamos como se comporta CLIP para clasificar dos set de datos con características distintas entre ellos.



## Dataset Food101

Primero probaremos en el dataset Food101. Este dataset consiste en 101 comidas diferentes y un total de 101.000 imágenes de estas.

In [ ]:
from torchvision.datasets import Food101

food_dataset = Food101(os.path.expanduser("~/.cache"), download=True)

### *Queries* para Predicción

Primero que todo, debemos construir las *queries* que utilizaremos para realizar la clasificación. Recordemos que el modelo que vamos a utilizar está entrenado para entregar una correspondencia entre un texto y una imágen. Por esto, para efectuar clasificaciones tendremos que entregarle la clase que queremos clasificar en forma de texto y para esto usaremos unas *queries* predefinidas.

A continuación, definiremos un *template* para construir nuestras *queries* y veremos algunos ejemplos de las que utilizaremos para clasificar el dataset de Food101.

In [ ]:
query_template = 'A photo of {}, a type of food.'
queries = [query_template.format(label) for label in food_dataset.classes]
tokenized_queries = clip.tokenize(queries).to(device)

In [ ]:
for query in queries[:10]:
    print(query)

### Visualización de Predicciones

Seleccione una imágen del dataset utilizando el parámetro `index` para ver las predicciones del modelo. Si prefiere elegir una imágen al azar, active el campo de `random_sample`.

In [ ]:
random_sample = True #@param {type:"boolean"}
index = 0 #@param {type:"integer"}


In [ ]:
if random_sample:
    index = random.randint(0, len(food_dataset))
original_image, true_label = food_dataset[index]
image = preprocess(original_image)
image_input = image.unsqueeze(0).to(device)

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features = model.encode_text(tokenized_queries).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
top_probs, top_labels = text_probs.cpu().topk(5, dim=-1)

Podemos explorar algunos de los resultados que genera CLIP, a continuación veremos las dimensiones de los vectores de características producidos, tanto de texto como de la imagen, además de la matriz de similaridades entre los distintos textos e imágenes que le entregamos.

In [ ]:
print('Dim features de la imágen   :', image_features.shape)
print('Dim features del texto      :', text_features.shape)
print('Dim matriz de similaridades :', text_probs.shape)

Veamos visualmente como es la predicción de este modelo para el ejemplo seleccionado.

In [ ]:
true_label_name = food_dataset.classes[true_label]

print(bold('Predicciones Top-5\n'))
print(bold(f'Clase          '), bold(f'Probabilidad'))
for prob, label in zip(top_probs[0].tolist(), top_labels[0].tolist()):
    label_name = food_dataset.classes[label]
    if true_label_name == label_name:
        print(bold(f'{label_name:15.15s} {prob:.4f}'))
    else:
        print(f'{label_name:15.15s} {prob:.4f}')

print(bold(f'\nClase verdadera: {true_label_name}'), end='\n\n')
original_image

### Rendimiento del Modelo

Ahora midamos el rendimiento que obtiene este modelo si lo usamos para clasificar en el set de test.

Es importante recordar que el modelo que estamos usando no fue entrenado para clasificar en este, por lo que estamos usando una técnica zero-shot.

In [ ]:
from tqdm.auto import tqdm
from torch.utils.data import DataLoader

def evaluate_model(model, dataset, queries, batch_size=512):
    test_loader = DataLoader(dataset, batch_size=batch_size)

    in_top1 = in_top5 = total = 0.
    total_batches = len(test_dataset) // batch_size
    for image_inputs, true_labels in tqdm(test_loader, total=total_batches):
        image_inputs = image_inputs.to(device)
        with torch.no_grad():
            image_features = model.encode_image(image_inputs).float()
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features = model.encode_text(queries).float()
            text_features /= text_features.norm(dim=-1, keepdim=True)

        text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        top_probs, top_labels = text_probs.cpu().topk(5, dim=-1)

        label_match = (top_labels == true_labels.unsqueeze(-1))
        in_top1 += float(label_match[:,0].sum())
        in_top5 += float(label_match.any(-1).sum())
        total += true_labels.numel()

    top1_acc = in_top1 / total
    top5_acc = in_top5 / total

    return top1_acc, top5_acc

In [ ]:
batch_size = 512
test_dataset = Food101(os.path.expanduser("~/.cache"),
                       split='test',
                       transform=preprocess,
                       download=True)

In [ ]:
top1_acc, top5_acc = evaluate_model(
    model, test_dataset, tokenized_queries, batch_size=512)

In [ ]:
top1_acc = top1_acc * 100
top5_acc = top5_acc * 100
print(f'Top-1 Accuracy: {top1_acc:.2f}%')
print(f'Top-5 Accuracy: {top5_acc:.2f}%')

### Actividad 3

Responda las siguientes preguntas

1. **En la celda en donde obtuvimos las dimensiones de distintos resultados generados por el modelo.**

A qué corresponde el valor de la última dimension de los features de texto e imágenes (512 en este caso)

In [ ]:
R = '' #@param {type:"string"}

¿Por qué la matriz de similaridad es de `1x101`?

In [ ]:
R = '' #@param {type:"string"}


2. **Sugiera 2 templates para queries distintos al utilizado previamente y testee su rendimiento.**

El formato de la query debe seguir el mismo que la utilizada previamente y debe cada una debe ser ingresada en un textbox distinto.

Para testear sus templates, simplemente descomente el código más abajo.



In [ ]:
Q1 = "" #@param {type:"string"}
Q2 = "" #@param {type:"string"}

In [ ]:
# ex1_queries = [Q1.format(label) for label in food_dataset.classes]
# ex1_tokenized_queries = clip.tokenize(ex1_queries).to(device)

# top1_acc, top5_acc = evaluate_model(
#     model, test_dataset, ex1_tokenized_queries, batch_size=512)

# top1_acc = top1_acc * 100
# top5_acc = top5_acc * 100
# print(f'Top-1 Accuracy: {top1_acc:.2f}%')
# print(f'Top-5 Accuracy: {top5_acc:.2f}%')

In [ ]:
# ex2_queries = [Q2.format(label) for label in food_dataset.classes]
# ex2_tokenized_queries = clip.tokenize(ex2_queries).to(device)

# top1_acc, top5_acc = evaluate_model(
#     model, test_dataset, ex2_tokenized_queries, batch_size=512)

# top1_acc = top1_acc * 100
# top5_acc = top5_acc * 100
# print(f'Top-1 Accuracy: {top1_acc:.2f}%')
# print(f'Top-5 Accuracy: {top5_acc:.2f}%')

## Dataset Stanford Cars

Ahora probaremos como le va al modelo en otro dataset, con objetos totalmente distintos a los del anterior. En este caso se trata de un datasets de automoviles. De igual manera que el anterior, el modelo no fue entrenado con estas imágenes en ningún momento.

In [ ]:
!mkdir -p data
if not os.path.exists('data/stanford_cars.zip'):
    !gdown --id 1JkcF--obwMvo2ZocIiKEli3EoE0e6ngt -O data/stanford_cars.zip

!unzip -nq data/\*.zip -d data

In [ ]:
from torchvision.datasets import StanfordCars

cars_dataset = StanfordCars("data/")

De igual manera que para el dataset anterior, debemos definir nuestras *queries*.

In [ ]:
query_template = 'A photo of {}'
queries = [query_template.format(label) for label in cars_dataset.classes]
tokenized_queries = clip.tokenize(queries).to(device)

In [ ]:
for query in queries[:10]:
    print(query)

### Visualización de Predicciones

Igual que con el dataset anterior, seleccione una imágen del dataset utilizando el parámetro `index` para ver las predicciones del modelo. Si prefiere elegir una imágen al azar, active el campo de `random_sample`.

In [ ]:
random_sample = True #@param {type:"boolean"}
index = 0 #@param {type:"integer"}


In [ ]:
if random_sample:
    index = random.randint(0, len(cars_dataset))
original_image, true_label = cars_dataset[index]
image = preprocess(original_image)
image_input = image.unsqueeze(0).to(device)

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features = model.encode_text(tokenized_queries).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
top_probs, top_labels = text_probs.cpu().topk(5, dim=-1)

Visualicemos el resultado del modelo.

In [ ]:
true_label_name = cars_dataset.classes[true_label]

print(bold('Predicciones Top-5\n'))
print(bold(f'Clase               '), bold(f'Probabilidad'))
for prob, label in zip(top_probs[0].tolist(), top_labels[0].tolist()):
    label_name = cars_dataset.classes[label]
    if true_label_name == label_name:
        print(bold(f'{label_name:20.20s} {prob:.4f}'))
    else:
        print(f'{label_name:20.20s} {prob:.4f}')

print(bold(f'\nClase verdadera: {true_label_name}'), end='\n\n')
original_image

### Rendimiento en Test

Ahora, evaluemos el modelo en el set de test del dataset Stanford Cars.

In [ ]:
from torch.utils.data import DataLoader

batch_size = 512

test_cars_dataset = StanfordCars('data/', split='test',
                                 transform=preprocess)
test_loader = DataLoader(test_cars_dataset, batch_size=batch_size)

In [ ]:
from tqdm.auto import tqdm

in_top1 = in_top5 = total = 0.
total_batches = len(test_cars_dataset) // batch_size
for image_inputs, true_labels in tqdm(test_loader, total=total_batches):
    image_inputs = image_inputs.to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_inputs).float()
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features = model.encode_text(tokenized_queries).float()
        text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    top_probs, top_labels = text_probs.cpu().topk(5, dim=-1)

    label_match = (top_labels == true_labels.unsqueeze(-1))
    in_top1 += float(label_match[:,0].sum())
    in_top5 += float(label_match.any(-1).sum())
    total += true_labels.numel()

In [ ]:
top1_acc = in_top1 / total * 100
top5_acc = in_top5 / total * 100
print(f'Top-1 Accuracy: {top1_acc:.2f}%')
print(f'Top-5 Accuracy: {top5_acc:.2f}%')

### Actividad 4

Prueba con tus propias imágenes. Utiliza el ceodigo a continuación para subir 5 imágenes distintas y generar 5 queries para estas, las queries deben ser distintas y debe haber una asociada a cada imagen. No necesariamente deben todas seguir el mismo *template*.

Escribe abajo un pequeño análisis del resultados obtenido.

In [ ]:
A = "" #@param {type:"string"}

In [ ]:
from google.colab import files

uploaded = files.upload()
filenames = list(uploaded.keys())

In [ ]:
from PIL import Image

images = [Image.open(fn) for fn in filenames]
image_inputs = torch.stack([preprocess(image) for image in images]).to(device)

In [ ]:
# queries = [
#     "An image of an apple",
#     "An image of an orange",
#     "etc...",
# ]
queries = [
    "A photo of apples, a type of fruit",
    "A photo of pears, a type of fruit",
    "A photo of papayas, a type of fruit",
    "A photo of a pineapple, a type of fruit",
    "A photo of a watermelon, a type of fruit",
]
tokenized_queries = clip.tokenize(queries).to(device)

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image_inputs).float()
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features = model.encode_text(tokenized_queries).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

similarity = (image_features @ text_features.T).softmax(dim=-1).cpu().numpy().T

In [ ]:
similarity.shape

In [ ]:
count = len(queries)

plt.figure(figsize=(20, 14))
plt.imshow(similarity, vmin=0.1, vmax=0.3)

plt.yticks(range(count), queries, fontsize=14)
plt.xticks([])
for i, image in enumerate(images):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")
for x in range(similarity.shape[1]):
    for y in range(similarity.shape[0]):
        plt.text(x, y, f"{similarity[y, x]:.2f}", ha="center", va="center", size=12)

for side in ["left", "top", "right", "bottom"]:
  plt.gca().spines[side].set_visible(False)

plt.xlim([-0.5, count - 0.5])
plt.ylim([count + 0.5, -2])

plt.title("Cosine similarity between text and image features", size=20)
plt.show()